In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
size = 256
images = []
i=0
for file in os.listdir("bolek"):
    image = cv2.imread(f"bolek/{file}")
    image = cv2.resize(image, (size,size))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)
images = np.array(images)
images = images.astype('float32') / 255.0

print('loaded training images',len(images))

In [ ]:
# create BW images
bwImages = []
for i in range(images.shape[0]):
    m = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
    bwImages.append(m)
bwImages = np.array(bwImages)
bwImages = np.expand_dims(bwImages,3)
print(bwImages.shape)

In [ ]:
import matplotlib.pyplot as plt
import random
plt.figure(figsize=(15, 15))
n = 4
x = 1
print("Black&White")
for i in range(n):
    ax = plt.subplot(1,n, x)
    x = x + 1
    plt.imshow(bwImages[i],cmap='gray')
    plt.axis("off")
plt.show()
plt.figure(figsize=(15, 15))
x = 1
print("Color")
for i in range(n):
    ax = plt.subplot(1,n, x)
    x = x + 1
    plt.imshow(images[i])
    plt.axis("off")
plt.show()

In [ ]:
import random
def show_images(bwImages,realImages,number_of_images=3,fig_size=(12,12)):
    n=number_of_images
    rnd = random.sample(range(realImages.shape[0]), n)

    plt.figure(figsize=fig_size)
    print('Black&White images')
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(bwImages[rnd[i]],cmap='gray')
        plt.axis("off")
    plt.show()

    resultImages = autoencoder.predict(bwImages)
    plt.figure(figsize=fig_size)
    print('Colorized BW images')
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(tf.clip_by_value(resultImages[rnd[i]], 0.0, 1.0))
        plt.axis("off")
    plt.show()

    plt.figure(figsize=fig_size)
    print('Original images')
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(realImages[rnd[i]])
        plt.axis("off")
    plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input
from tensorflow.keras.layers import UpSampling2D, LeakyReLU, Conv2DTranspose, Concatenate
from tensorflow.keras.initializers import RandomNormal

def colorize_model(image_shape):
    conv_args = {"activation": "relu","padding": "same", "kernel_size": (3,3)}
    input_img = Input(shape=image_shape)
    x = Conv2D(filters = 16, **conv_args)(input_img)
    x = Conv2D(filters = 32, **conv_args)(x)
    x = Conv2D(filters = 64, **conv_args)(x)
    x = Dense(10)(x)
    output_img = Conv2D(3, (3, 3),  padding='same')(x)
    model = Model(input_img, output_img)
    return model

import autoencoder_models
#autoencoder = autoencoder_models.unet_model((None,None,3))
autoencoder = colorize_model((None,None,1))

autoencoder.summary()
optimizer = tf.keras.optimizers.Adam()
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')


In [ ]:
# add two channels to BW images - necessary for out UNET model!
print(bwImages.shape)
bwImages = np.concatenate((bwImages,bwImages,bwImages),axis=3)
print(bwImages.shape)

In [ ]:
for epoch in range(30):
    autoencoder.fit(bwImages, images, batch_size=2, epochs=1)
    if epoch%1==0:
        print("Epoch",epoch)
        show_images(bwImages,images)
    

In [ ]:

image = cv2.imread(f"bolek/frame1501.jpg")
#    image = cv2.resize(image, (size,size))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
imageBW = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
imageBW = np.expand_dims(imageBW,2)

image = np.expand_dims(image,0)
imageBW = np.expand_dims(imageBW,0)

image = image/255.0
imageBW = imageBW/255.0
#imageBW = np.concatenate((imageBW,imageBW,imageBW),axis=3)
show_images(imageBW,image,1)